## OpenAI Embedding
* api 키를 발급받아 사용

In [1]:
%pip install openai

  Using cached anyio-4.6.0-py3-none-any.whl.metadata (4.6 kB)
  Using cached httpx-0.27.2-py3-none-any.whl.metadata (7.1 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached h11-0.14.0-py3-none-any.whl.metadata (8.2 kB)
Using cached anyio-4.6.0-py3-none-any.whl (89 kB)
Using cached httpx-0.27.2-py3-none-any.whl (76 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 28.9 MB/s eta 0:00:00
Using cached sniffio-1.3.1-py3-none-any.whl (10 kB)
Using cached h11-0.14.0-py3-none-any.whl (58 kB)
Note: you may need to restart the kernel to use updated packages.


In [3]:
from openai import OpenAI

# API_KEY =  your key here

client = OpenAI(api_key=API_KEY)

In [5]:
embedding = client.embeddings.create(
    model="text-embedding-ada-002", input="야 저기 차 온다!"
)

len(embedding.data[0].embedding)

1536

In [6]:
import pandas as pd

data = [
    "내일 차 타고 놀러가자",
    "지금 오는 버스는 어디서 오는 버스야?",
    "차 한잔 하면서 이야기 하시죠",
    "5차 공동구매! 오늘만 세일!",
    "홍차 녹차 중에 어떤 차가 좋으세요?",
]

df = pd.DataFrame(data, columns=["text"])
df

,text
0,내일 차 타고 놀러가자
1,지금 오는 버스는 어디서 오는 버스야?
2,차 한잔 하면서 이야기 하시죠
3,5차 공동구매! 오늘만 세일!
4,홍차 녹차 중에 어떤 차가 좋으세요?


In [7]:
def get_embedding(text):
    response = client.embeddings.create(input=text, model="text-embedding-ada-002")
    return response.data[0].embedding

In [8]:
df["embedding"] = df.apply(lambda row: get_embedding(row.text), axis=1)
df

,text,embedding
0,내일 차 타고 놀러가자,"[0.015895135700702667, -0.011393633671104908, ..."
1,지금 오는 버스는 어디서 오는 버스야?,"[0.008037855848670006, -0.011817412450909615, ..."
2,차 한잔 하면서 이야기 하시죠,"[0.016478225588798523, -0.02388259395956993, 0..."
3,5차 공동구매! 오늘만 세일!,"[-0.017510101199150085, -0.002525998977944255,..."
4,홍차 녹차 중에 어떤 차가 좋으세요?,"[0.005494758952409029, -0.011675575748085976, ..."


## 유사도 구하기

In [9]:
import numpy as np


def cos_sim(A, B):
    return A @ B / (np.linalg.norm(A) * np.linalg.norm(B))


def return_answer_candidate(df, query):
    """
    df : 문장과 임베딩 벡터가 들어있는 데이터 프레임
    query : 질의할 문장
    """
    query_embedding = get_embedding(query)
    df["similarity"] = df.embedding.apply(
        lambda x: cos_sim(np.array(x), np.array(query_embedding))
    )

    results_co = df.sort_values("similarity", ascending=False, ignore_index=True)
    return results_co.head(3)

In [10]:
sim_result = return_answer_candidate(df, "야 저기 차 온다")
sim_result

,text,embedding,similarity
0,내일 차 타고 놀러가자,"[0.015895135700702667, -0.011393633671104908, ...",0.896481
1,차 한잔 하면서 이야기 하시죠,"[0.016478225588798523, -0.02388259395956993, 0...",0.875385
2,지금 오는 버스는 어디서 오는 버스야?,"[0.008037855848670006, -0.011817412450909615, ...",0.848097


In [11]:
sim_result = return_answer_candidate(df, "예쁜 카페가고 싶어")
sim_result

,text,embedding,similarity
0,내일 차 타고 놀러가자,"[0.015895135700702667, -0.011393633671104908, ...",0.822717
1,차 한잔 하면서 이야기 하시죠,"[0.016478225588798523, -0.02388259395956993, 0...",0.812945
2,홍차 녹차 중에 어떤 차가 좋으세요?,"[0.005494758952409029, -0.011675575748085976, ...",0.812300
